In [ ]:
import sys
import os

module_path = os.path.abspath(os.path.join('..', 'utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

import format_time as ft
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from adjustText import adjust_text


EPOCHS = 100
BATCH_SIZE = 128
SAMPLES = 50000

FIGURES_PATH = '../figures/'
isExist = os.path.exists(FIGURES_PATH)
if not isExist:
    os.makedirs(FIGURES_PATH)

RESULTS_PATH = '../results_all/'
# SERVER_PATH = RESULTS_PATH + 'server_3090/results_100epochs_128batch/'
SERVER_PATH = RESULTS_PATH + 'server_a2000/results_100epochs_128batch/'
# SERVER_PATH = RESULTS_PATH + 'server_3080/results_100epochs_128batch/'
path_elements = SERVER_PATH.split('/')
server_element = [element for element in path_elements if 'server' in element]
SERVER_NAME = server_element[0]


df_nostats_train = pd.read_csv(SERVER_PATH + r'mlresults_nostats_train.csv')
df_nostats_train['duration(s)'] = df_nostats_train['duration(s)'].apply(lambda x: ft.plot_time(x))
df_nostats_train['step(ms)'] = df_nostats_train['step(ms)'].apply(lambda x: ft.plot_time(x))

df_stats_train = pd.read_csv(SERVER_PATH + r'mlresults_stats_train.csv')
df_stats_train['duration(s)'] = df_stats_train['duration(s)'].apply(lambda x: ft.plot_time(x))
df_stats_train['step(ms)'] = df_stats_train['step(ms)'].apply(lambda x: ft.plot_time(x))

df_eco2ai_train = pd.read_csv(SERVER_PATH + r'mlresults_eco2ai_nostats_train.csv')
df_eco2ai_train['duration(s)'] = df_eco2ai_train['duration(s)'].apply(lambda x: ft.plot_time(x))
df_eco2ai_train['step(ms)'] = df_eco2ai_train['step(ms)'].apply(lambda x: ft.plot_time(x))

df_codecarbon_train = pd.read_csv(SERVER_PATH + r'mlresults_codecarbon_nostats_train.csv')
df_codecarbon_train['duration(s)'] = df_codecarbon_train['duration(s)'].apply(lambda x: ft.plot_time(x))
df_codecarbon_train['step(ms)'] = df_codecarbon_train['step(ms)'].apply(lambda x: ft.plot_time(x))

df_carbontracker_train = pd.read_csv(SERVER_PATH + r'mlresults_carbontracker_nostats_train.csv')
df_carbontracker_train['duration(s)'] = df_carbontracker_train['duration(s)'].apply(lambda x: ft.plot_time(x))
df_carbontracker_train['step(ms)'] = df_carbontracker_train['step(ms)'].apply(lambda x: ft.plot_time(x))

df_nostats_test = pd.read_csv(SERVER_PATH + r'mlresults_nostats_test.csv')
df_nostats_test['duration(s)'] = df_nostats_test['duration(s)'].apply(lambda x: ft.plot_time(x))
df_nostats_test['step(ms)'] = df_nostats_test['step(ms)'].apply(lambda x: ft.plot_time(x))

df_stats_test = pd.read_csv(SERVER_PATH + r'mlresults_stats_test.csv')
df_stats_test['duration(s)'] = df_stats_test['duration(s)'].apply(lambda x: ft.plot_time(x))
df_stats_test['step(ms)'] = df_stats_test['step(ms)'].apply(lambda x: ft.plot_time(x))

df_eco2ai_test = pd.read_csv(SERVER_PATH + r'mlresults_eco2ai_nostats_test.csv')
df_eco2ai_test['duration(s)'] = df_eco2ai_test['duration(s)'].apply(lambda x: ft.plot_time(x))
df_eco2ai_test['step(ms)'] = df_eco2ai_test['step(ms)'].apply(lambda x: ft.plot_time(x))

df_codecarbon_test = pd.read_csv(SERVER_PATH + r'mlresults_codecarbon_nostats_test.csv')
df_codecarbon_test['duration(s)'] = df_codecarbon_test['duration(s)'].apply(lambda x: ft.plot_time(x))
df_codecarbon_test['step(ms)'] = df_codecarbon_test['step(ms)'].apply(lambda x: ft.plot_time(x))

df_carbontracker_test = pd.read_csv(SERVER_PATH + r'mlresults_carbontracker_nostats_test.csv')
df_carbontracker_test['duration(s)'] = df_carbontracker_test['duration(s)'].apply(lambda x: ft.plot_time(x))
df_carbontracker_test['step(ms)'] = df_carbontracker_test['step(ms)'].apply(lambda x: ft.plot_time(x))

df_codecarbon = pd.read_csv(SERVER_PATH + r'emissions.csv')
df_codecarbon_results_train = pd.DataFrame(df_codecarbon.values[::2],index=df_codecarbon.index[::2],columns=df_codecarbon.columns).reset_index(drop=True)
df_codecarbon_results_test = pd.DataFrame(df_codecarbon.values[1::2],index=df_codecarbon.index[1::2],columns=df_codecarbon.columns).reset_index(drop=True)

df_eco2ai = pd.read_csv(SERVER_PATH + r'resultsEco2AI.csv')
df_eco2ai_results_train = pd.DataFrame(df_eco2ai.values[::2],index=df_eco2ai.index[::2],columns=df_eco2ai.columns).reset_index(drop=True)
df_eco2ai_results_test = pd.DataFrame(df_eco2ai.values[1::2],index=df_eco2ai.index[1::2],columns=df_eco2ai.columns).reset_index(drop=True)

with open(SERVER_PATH + 'stats_train.json') as f:
    stats_train = json.load(f)
stats_train = stats_train['exp_0']

with open(SERVER_PATH + 'stats_test.json') as f:
    stats_test = json.load(f)
stats_test = stats_test['exp_0']

with open(SERVER_PATH + 'model_size.json') as f:
    model_size = json.load(f)

def add_label(x, y, val, ax, distance=0, other=None):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    texts = []
    for i, point in a.iterrows():
        if other is None:
            texts.append(ax.text(point['x']+distance, point['y'], str(point['val'])))
        else:
            texts.append(ax.text(point['x']+distance, point['y'], str(point['val']) + str(other)))
    adjust_text(texts)

In [ ]:
sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="accuracy", data=df_stats_test, estimator=np.max, errorbar=None)
plt.xticks(rotation=45)
plt.title('Model Max Accuracy')
plt.xlabel("Models")
plt.ylabel("Accuracy (%)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(),
            '{:.2f}%'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom')

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/max_model_accuracy_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="duration(s)", data=df_stats_train, estimator=np.mean, errorbar=('sd',3))
plt.xticks(rotation=45)
plt.title('Model Average Training Duration per Epoch (std of +/-3)')
plt.xlabel("Models")
plt.ylabel("Duration (ms)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(),
            '{:.0f}ms'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom')

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/model_training_per_epoch_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="duration(s)", data=df_stats_test, estimator=np.mean, errorbar=('sd',3))
plt.xticks(rotation=45)
plt.title('Model Average Testing Duration per 50k samples (std of +/-3)')
plt.xlabel("Models")
plt.ylabel("Duration (ms)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height()+100,
            '{:.0f}ms'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom')

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/model_testing_per_50ksamples_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_duration_df = pd.merge(df_nostats_train, df_stats_train, on=['project_name', 'epoch'], suffixes=('_nostats', '_frost'))
merged_duration_df.rename(columns={"duration(s)_nostats": "NoTool"}, inplace=True)
merged_duration_df.rename(columns={"duration(s)_frost": "FROST"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_eco2ai_train, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"duration(s)": "Eco2AI"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_codecarbon_train, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"duration(s)": "CodeCarbon"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_carbontracker_train, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"duration(s)": "CarbonTracker"}, inplace=True)
merged_duration_df = merged_duration_df[['project_name', 'NoTool', 'FROST', 'Eco2AI', 'CodeCarbon', 'CarbonTracker']]

sns.set_theme(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid", font_scale=2.5)

sns.catplot(x="project_name", y="value", hue="variable", kind="bar", errorbar=lambda x: (x.min(), x.max()), data=pd.melt(merged_duration_df, id_vars=['project_name']), height=16, aspect=2/1, legend_out=False)
plt.xticks(rotation=45)
plt.title('Runtime (training)')
plt.xlabel("Models")
plt.ylabel("Milliseconds (ms)")
plt.legend(loc='upper right', title='Experiment')
plt.tight_layout()
plt.savefig(FIGURES_PATH + '/runtime_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

merged_duration_df = pd.merge(df_nostats_train, df_stats_train, on=['project_name', 'epoch'], suffixes=('_nostats', '_frost'))
merged_duration_df.rename(columns={"step(ms)_nostats": "NoTool"}, inplace=True)
merged_duration_df.rename(columns={"step(ms)_frost": "FROST"}, inplace=True)

merged_duration_df = pd.merge(merged_duration_df, df_eco2ai_train, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"step(ms)": "Eco2AI"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_codecarbon_train, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"step(ms)": "CodeCarbon"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_carbontracker_train, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"step(ms)": "CarbonTracker"}, inplace=True)
merged_duration_df = merged_duration_df[['project_name', 'NoTool', 'FROST', 'Eco2AI', 'CodeCarbon', 'CarbonTracker']]

sns.set_theme(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid", font_scale=2.5)

sns.catplot(x="project_name", y="value", hue="variable", kind="bar", errorbar=lambda x: (x.min(), x.max()), data=pd.melt(merged_duration_df, id_vars=['project_name']), height=16, aspect=2/1, legend_out=False)
plt.xticks(rotation=45)
plt.title('Runtime per batch (training)')
plt.xlabel("Models")
plt.ylabel("Milliseconds (ms)")
plt.legend(loc='upper right', title='Experiment')
plt.tight_layout()
plt.savefig(FIGURES_PATH + '/runtime_per_batch_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_duration_df = pd.merge(df_nostats_test, df_stats_test, on=['project_name', 'epoch'], suffixes=('_nostats', '_frost'))
merged_duration_df.rename(columns={"duration(s)_nostats": "NoTool"}, inplace=True)
merged_duration_df.rename(columns={"duration(s)_frost": "FROST"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_eco2ai_test, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"duration(s)": "Eco2AI"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_codecarbon_test, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"duration(s)": "CodeCarbon"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_carbontracker_test, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"duration(s)": "CarbonTracker"}, inplace=True)
merged_duration_df = merged_duration_df[['project_name', 'NoTool', 'FROST', 'Eco2AI', 'CodeCarbon', 'CarbonTracker']]

sns.set_theme(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid", font_scale=2.5)

sns.catplot(x="project_name", y="value", hue="variable", kind="bar", errorbar=lambda x: (x.min(), x.max()), data=pd.melt(merged_duration_df, id_vars=['project_name']), height=16, aspect=2/1, legend_out=False)
plt.xticks(rotation=45)
plt.title('Runtime (testing)')
plt.xlabel("Models")
plt.ylabel("Milliseconds (ms)")
plt.legend(loc='upper right', title='Experiment')
plt.savefig(FIGURES_PATH + '/runtime_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.tight_layout()
plt.show()

merged_duration_df = pd.merge(df_nostats_test, df_stats_test, on=['project_name', 'epoch'], suffixes=('_nostats', '_frost'))
merged_duration_df.rename(columns={"step(ms)_nostats": "NoTool"}, inplace=True)
merged_duration_df.rename(columns={"step(ms)_frost": "FROST"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_eco2ai_test, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"step(ms)": "Eco2AI"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_codecarbon_test, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"step(ms)": "CodeCarbon"}, inplace=True)
merged_duration_df = pd.merge(merged_duration_df, df_carbontracker_test, on=['project_name', 'epoch'])
merged_duration_df.rename(columns={"step(ms)": "CarbonTracker"}, inplace=True)
merged_duration_df = merged_duration_df[['project_name', 'NoTool', 'FROST', 'Eco2AI', 'CodeCarbon', 'CarbonTracker']]

sns.set_theme(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid", font_scale=2.5)

sns.catplot(x="project_name", y="value", hue="variable", kind="bar", errorbar=lambda x: (x.min(), x.max()), data=pd.melt(merged_duration_df, id_vars=['project_name']), height=16, aspect=2/1, legend_out=False)
plt.xticks(rotation=45)
plt.title('Runtime per batch (testing)')
plt.xlabel("Models")
plt.ylabel("Milliseconds (ms)")
plt.legend(loc='upper right', title='Experiment')
plt.tight_layout()
plt.savefig(FIGURES_PATH + '/runtime_per_batch_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
stats_train_tmp = stats_train.copy()

models = df_stats_train.project_name.unique()

for model in models:
    tmp_dict = stats_train_tmp[model]
    if model in model_size.keys():
        model_size_tmp = model_size[model]
        for epoch in tmp_dict.keys():
            filtered = filter(lambda num: num >= 0, tmp_dict[epoch]['cpu_delta_power_w'])
            tmp_dict[epoch]['cpu_delta_power_w'] = list(filtered)

            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'start_time'] = tmp_dict[epoch]['start_time']
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'stop_time'] = tmp_dict[epoch]['stop_time']
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'cpu_energy_uj'] = np.mean(tmp_dict[epoch]['cpu_energy_uj'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'cpu_delta_power_w'] = np.mean(tmp_dict[epoch]['cpu_delta_power_w'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'cpu_percent'] = np.mean(tmp_dict[epoch]['cpu_percent'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'cpu_memory_percent'] = np.mean(tmp_dict[epoch]['cpu_memory_percent'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'cpu_temperature_c'] = np.mean(tmp_dict[epoch]['cpu_temperature_c'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'gpu_power_w'] = np.mean(tmp_dict[epoch]['gpu_power_w'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'gpu_temperature_c'] = np.mean(tmp_dict[epoch]['gpu_temperature_c'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'gpu_memory_free_b'] = np.mean(tmp_dict[epoch]['gpu_memory_free_b'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'gpu_memory_used_b'] = np.mean(tmp_dict[epoch]['gpu_memory_used_b'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'gpu_percent'] = np.mean(tmp_dict[epoch]['gpu_percent'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'ram_power_w'] = np.mean(tmp_dict[epoch]['ram_power_w'])
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'size_mb'] = model_size_tmp['size_mb']
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'parameters'] = model_size_tmp['parameters']
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'buffer'] = model_size_tmp['buffer']
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'macs'] = model_size_tmp['macs']
            df_stats_train.loc[(df_stats_train['project_name'] == model) & (df_stats_train['epoch'] == int(epoch)), 'trainable_params'] = model_size_tmp['trainable_params']

stats_test_tmp = stats_test.copy()

models = df_stats_test.project_name.unique()

for model in models:
    tmp_dict = stats_test_tmp[model]
    if model in model_size.keys():
        model_size_tmp = model_size[model]
        for epoch in tmp_dict.keys():
            filtered = filter(lambda num: num >= 0, tmp_dict[epoch]['cpu_delta_power_w'])
            tmp_dict[epoch]['cpu_delta_power_w'] = list(filtered)
            
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'start_time'] = tmp_dict[epoch]['start_time']
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'stop_time'] = tmp_dict[epoch]['stop_time']
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'cpu_energy_uj'] = np.mean(tmp_dict[epoch]['cpu_energy_uj'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'cpu_delta_power_w'] = np.mean(tmp_dict[epoch]['cpu_delta_power_w'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'cpu_percent'] = np.mean(tmp_dict[epoch]['cpu_percent'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'cpu_memory_percent'] = np.mean(tmp_dict[epoch]['cpu_memory_percent'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'cpu_temperature_c'] = np.mean(tmp_dict[epoch]['cpu_temperature_c'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'gpu_power_w'] = np.mean(tmp_dict[epoch]['gpu_power_w'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'gpu_temperature_c'] = np.mean(tmp_dict[epoch]['gpu_temperature_c'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'gpu_memory_free_b'] = np.mean(tmp_dict[epoch]['gpu_memory_free_b'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'gpu_memory_used_b'] = np.mean(tmp_dict[epoch]['gpu_memory_used_b'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'gpu_percent'] = np.mean(tmp_dict[epoch]['gpu_percent'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'ram_power_w'] = np.mean(tmp_dict[epoch]['ram_power_w'])
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'size_mb'] = model_size_tmp['size_mb']
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'parameters'] = model_size_tmp['parameters']
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'buffer'] = model_size_tmp['buffer']
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'macs'] = model_size_tmp['macs']
            df_stats_test.loc[(df_stats_test['project_name'] == model) & (df_stats_test['epoch'] == int(epoch)), 'trainable_params'] = model_size_tmp['trainable_params']        

In [ ]:
sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="cpu_delta_power_w", data=df_stats_train, estimator= np.max, errorbar=None)
plt.xticks(rotation=45)
plt.title('Average CPU power consumption (training)')
plt.xlabel("Models")
plt.ylabel("Average CPU power (W)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(),
            '{:.2f}W'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/avg_cpu_power_consumption_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="cpu_delta_power_w", data=df_stats_test, estimator= np.max, errorbar=None)
plt.xticks(rotation=45)
plt.title('Average CPU power consumption (testing)')
plt.xlabel("Models")
plt.ylabel("Average CPU power (W)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(),
            '{:.2f}W'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/avg_cpu_power_consumption_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="gpu_power_w", data=df_stats_train, estimator=lambda x: np.mean(x)/1000)
plt.xticks(rotation=45)
plt.title('Average GPU power consumption (training)')
plt.xlabel("Models")
plt.ylabel("Average GPU power (W)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(),
            '{:.2f}W'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/avg_gpu_power_consumption_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
ax = sns.barplot(x="project_name", y="gpu_power_w", data=df_stats_test, estimator=lambda x: np.mean(x)/1000)
plt.xticks(rotation=45)
plt.title('Average GPU power consumption (testing)')
plt.xlabel("Models")
plt.ylabel("Average GPU power (W)")

# Adding text labels above the bars
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(),
            '{:.2f}W'.format(p.get_height()), # You can format it as you like
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/avg_gpu_power_consumption_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['gpu_power_w'] = round(df_stats_train_tmp['gpu_power_w']/1000, 2)

sns.set(rc={'figure.figsize':(11.7,5.27)},style="whitegrid")
sns.scatterplot(x="gpu_power_w", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Average GPU power consumption as a function of the utilisation (training)')
plt.xlabel("Average GPU power (W)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='upper left', title='Experiment')

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_power_vs_utilisation_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['gpu_power_w'] = round(df_stats_train_tmp['gpu_power_w']/1000, 2)

sns.set(rc={'figure.figsize':(11.7,5.27)},style="whitegrid")
sns.scatterplot(x="gpu_power_w", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Average GPU power consumption as a function of the utilisation (training)')
plt.xlabel("Average GPU power (W)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='upper left', title='Experiment')
plt.xlim(60, None)
plt.ylim(80, None)
# add_label(df_stats_train_tmp.gpu_power_w, df_stats_train_tmp.gpu_percent, df_stats_train_tmp.project_name, plt.gca(), 0.02)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_power_vs_utilisation_training_zoom_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['cpu_delta_power_w'] = round(df_stats_train_tmp['cpu_delta_power_w'],2)

sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
sns.scatterplot(x="cpu_delta_power_w", y="cpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Average CPU power consumption as a function of the utilisation (training)')
plt.xlabel("Average CPU power (W)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='upper left', title='Experiment')
plt.tight_layout()
plt.savefig(FIGURES_PATH + '/cpu_power_vs_utilisation_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['gpu_power_w'] = round(df_stats_test_tmp['gpu_power_w']/1000, 2)

sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
sns.scatterplot(x="gpu_power_w", y="gpu_percent", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('Average GPU power consumption as a function of the utilisation (testing)')
plt.xlabel("Average GPU power (W)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='upper left', title='Experiment')
plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_power_vs_utilisation_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['cpu_delta_power_w'] = round(df_stats_test_tmp['cpu_delta_power_w'],2)

sns.set(rc={'figure.figsize':(11.7,5.27)}, style="whitegrid")
sns.scatterplot(x="cpu_delta_power_w", y="cpu_percent", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('Average CPU power consumption as a function of the utilisation (testing)')
plt.xlabel("Average CPU power (W)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='upper left', title='Experiment')
plt.tight_layout()
plt.savefig(FIGURES_PATH + '/cpu_power_vs_utilisation_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['total'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000*EPOCHS / 3600)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
sns.scatterplot(x="total", y="accuracy", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Total power consumption as a function of the accuracy (training)')
plt.xlabel("Total power consumption (Wh)")
plt.ylabel("Accuracy (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_train_tmp.total, df_stats_train_tmp.accuracy, df_stats_train_tmp.project_name, plt.gca(), 2)
plt.tight_layout() 
plt.savefig(FIGURES_PATH + '/total_power_vs_accuracy_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['gpu_power_w'] = round(df_stats_train_tmp['gpu_power_w']/1000, 2)
df_stats_train_tmp['gpu_memory_used_b'] = round(df_stats_train_tmp['gpu_memory_used_b']/pow(10, 9), 2)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="gpu_power_w", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', size='gpu_memory_used_b', sizes=(50, 500))
h,l = ax.get_legend_handles_labels()
plt.legend(h[1:15],l[1:15], loc='lower right', title='Experiment')
plt.title('Average GPU power consumption - dot size represents the GPU RAM usage (training)')
plt.xlabel("Average GPU power (W)")
plt.ylabel("Utilisation (%)")
plt.ylim(0, 100)
for index, row in df_stats_train_tmp.iterrows():
    ax.annotate(f"{row['gpu_memory_used_b']}GB", (row['gpu_power_w'] + 0.2, row['gpu_percent']))

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_power_vs_utilisation_vs_gpu_ram_use_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="gpu_power_w", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', size='gpu_memory_used_b', sizes=(50, 500))
h,l = ax.get_legend_handles_labels()
plt.legend(h[1:15],l[1:15], loc='lower right', title='Experiment')
plt.title('Average GPU power consumption - dot size represents the GPU RAM usage (training)')
plt.xlabel("Average GPU power (W)")
plt.ylabel("Utilisation (%)")
plt.xlim(66, None)
plt.ylim(88, 100)

add_label(df_stats_train_tmp.gpu_power_w, df_stats_train_tmp.gpu_percent, df_stats_train_tmp.gpu_memory_used_b, plt.gca(), 0.01, 'GB') 

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_power_vs_utilisation_vs_gpu_ram_use_training_zoom_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['size_mb'] = round(df_stats_train_tmp['size_mb'], 1)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="size_mb", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by model size (training)')
plt.xlabel("Model size (MB)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_train_tmp.size_mb, df_stats_train_tmp.gpu_percent, df_stats_train_tmp.project_name, plt.gca(), 0.5)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_model_size_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['size_mb'] = round(df_stats_test_tmp['size_mb'], 1)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="size_mb", y="gpu_percent", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by model size (testing)')
plt.xlabel("Model size (MB)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_test_tmp.size_mb, df_stats_test_tmp.gpu_percent, df_stats_test_tmp.project_name, plt.gca(), 1)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_model_size_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['parameters'] = df_stats_train_tmp['parameters']/pow(10, 6)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="parameters", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by no. of parameters (training)')
plt.xlabel("Parameters (no. in millions)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_train_tmp.parameters, df_stats_train_tmp.gpu_percent, df_stats_train_tmp.project_name, plt.gca(), 0.2)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_no_parameters_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['parameters'] = df_stats_test_tmp['parameters']/pow(10, 6)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="parameters", y="gpu_percent", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by no. of parameters (testing)')
plt.xlabel("Parameters (no. in millions)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_test_tmp.parameters, df_stats_test_tmp.gpu_percent, df_stats_test_tmp.project_name, plt.gca(), 0.2)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_no_parameters_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['trainable_params'] = df_stats_train_tmp['trainable_params']/pow(10, 6)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="trainable_params", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by trainable parameters (training)')
plt.xlabel("Parameters (no. in millions)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_train_tmp.trainable_params, df_stats_train_tmp.gpu_percent, df_stats_train_tmp.project_name, plt.gca(), 0.2)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_no_trainable_parameters_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['trainable_params'] = df_stats_test_tmp['trainable_params']/pow(10, 6)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="trainable_params", y="gpu_percent", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by trainable parameters (training)')
plt.xlabel("Parameters (no. in millions)")
plt.ylabel("Utilisation (%)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_test_tmp.trainable_params, df_stats_test_tmp.gpu_percent, df_stats_test_tmp.project_name, plt.gca(), 0.2)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_no_trainable_parameters_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['macs'] = df_stats_train_tmp['macs']/pow(10, 6)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="macs", y="gpu_percent", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by MAC operations (training)')
plt.xlabel("Parameters (no. in millions)")
plt.ylabel("Multiply-Accumulate Operations (no. in millions)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_train_tmp.macs, df_stats_train_tmp.gpu_percent, df_stats_train_tmp.project_name, plt.gca(), 0.2)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_macs_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['macs'] = df_stats_test_tmp['macs']/pow(10, 6)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="macs", y="gpu_percent", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('GPU Utilisation by MAC operations (training)')
plt.xlabel("Parameters (no. in millions)")
plt.ylabel("Multiply-Accumulate Operations (no. in millions)")
plt.legend(loc='lower right', title='Experiment')
add_label(df_stats_test_tmp.macs, df_stats_test_tmp.gpu_percent, df_stats_test_tmp.project_name, plt.gca(), 0.2)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_vs_macs_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['macs'] = df_stats_train_tmp['macs']/pow(10, 6)
df_stats_train_tmp['size_mb'] = round(df_stats_train_tmp['size_mb'], 1)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="size_mb", y="macs", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Model size as a function of the MAC operations')
plt.xlabel("Model Size (MB)")
plt.ylabel("Multiply-Accumulate Operations (no. in millions)")
plt.legend(loc='upper right', title='Experiment')
add_label(df_stats_train_tmp.size_mb, df_stats_train_tmp.macs, df_stats_train_tmp.project_name, plt.gca(), 1)

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/model_size_vs_macs_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_df = df_stats_train.groupby('project_name').mean().reset_index()
merged_df = merged_df[['project_name', 'gpu_percent']]
merged_df = pd.merge(merged_df, df_stats_test.groupby('project_name').mean().reset_index(), on=['project_name'])
merged_df = merged_df[['project_name', 'gpu_percent_y', 'gpu_percent_x']]
merged_df.rename(columns={'gpu_percent_x': 'gpu_percent_train', 'gpu_percent_y': 'gpu_percent_test'}, inplace=True)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
ax = sns.scatterplot(x="gpu_percent_train", y="gpu_percent_test", hue="project_name", data=merged_df, style='project_name', s=150)
plt.title('GPU Utilisation for training and testing')
plt.xlabel("Utilisation - training (%)")
plt.ylabel("Utilisation - testing (%)")
plt.legend(loc='upper left', title='Experiment')

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/gpu_utilisation_training_vs_testing' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_df = df_stats_train.groupby('project_name').mean().reset_index()
merged_df['gpu_power_w_train'] = round(merged_df['gpu_power_w']/1000, 2)
merged_df['gpu_memory_used_b_train'] = round(merged_df['gpu_memory_used_b']/pow(10, 9), 2)
merged_df = merged_df[['project_name', 'gpu_power_w_train', 'gpu_memory_used_b_train']]
merged_df = pd.merge(merged_df, df_stats_test.groupby('project_name').mean().reset_index(), on=['project_name'])
merged_df['gpu_power_w_testing'] = round(merged_df['gpu_power_w']/1000, 2)
merged_df['gpu_memory_used_b_testing'] = round(merged_df['gpu_memory_used_b']/pow(10, 9), 2)
merged_df = merged_df[['project_name', 'gpu_power_w_train', 'gpu_memory_used_b_train', 'gpu_power_w_testing', 'gpu_memory_used_b_testing']]

sns.set(style="whitegrid")
ax = sns.scatterplot(x="gpu_power_w_train", y="gpu_power_w_testing", hue="project_name", data=merged_df, style='project_name', s=150)
plt.title('GPU power consumption for training and testing')
plt.xlabel("GPU Power - training (Wh)")
plt.ylabel("GPU Power - testing (Wh)")
plt.legend(loc='upper left', title='Experiment')

plt.savefig(FIGURES_PATH + '/gpu_power_training_vs_testing' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_df = df_stats_train.groupby('project_name').mean().reset_index()
merged_df['total_training'] = (merged_df['cpu_delta_power_w'] + (merged_df['gpu_power_w']/1000) + merged_df['ram_power_w']) * (merged_df['duration(s)']/1000*EPOCHS / 3600)
merged_df = merged_df[['project_name', 'total_training']]
merged_df = pd.merge(merged_df, df_stats_test.groupby('project_name').mean().reset_index(), on=['project_name'])
merged_df['total_testing'] = (merged_df['cpu_delta_power_w'] + (merged_df['gpu_power_w']/1000) + merged_df['ram_power_w']) * (merged_df['duration(s)']/1000*EPOCHS / 3600)
merged_df = merged_df[['project_name', 'total_training', 'total_testing']]

sns.set(style="whitegrid")
ax = sns.scatterplot(x="total_training", y="total_testing", hue="project_name", data=merged_df, style='project_name', s=150)
plt.title('Total power consumption for training and testing')
plt.xlabel("Total Power - training (Wh)")
plt.ylabel("Total Power - testing (Wh)")
plt.legend(loc='upper left', title='Experiment')
add_label(merged_df.total_training, merged_df.total_testing, merged_df.project_name, plt.gca(), 1)

plt.savefig(FIGURES_PATH + '/total_power_training_vs_testing' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['total'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000*EPOCHS / 3600)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
sns.scatterplot(x="total", y="size_mb", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Total power consumption as a function of the model size (training)')
plt.xlabel("Total power consumption (Wh)")
plt.ylabel("Model size (MB)")
plt.xlim(0, None)
plt.ylim(0, 80)
plt.legend(loc='upper right', title='Experiment')
add_label(df_stats_train_tmp.total, df_stats_train_tmp.size_mb, df_stats_train_tmp.project_name, plt.gca(), 2)

plt.tight_layout() 
plt.savefig(FIGURES_PATH + '/total_power_vs_model_size_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['total'] = (df_stats_test_tmp['cpu_delta_power_w'] + (df_stats_test_tmp['gpu_power_w']/1000) + df_stats_test_tmp['ram_power_w']) * (df_stats_test_tmp['duration(s)']/1000*EPOCHS / 3600)

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
sns.scatterplot(x="total", y="size_mb", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('Total power consumption as a function of the model size (testing)')
plt.xlabel("Total power consumption (Wh)")
plt.ylabel("Model size (MB)")
plt.xlim(0, None)
plt.ylim(0, 80)
plt.legend(loc='upper right', title='Experiment')
add_label(df_stats_test_tmp.total, df_stats_test_tmp.size_mb, df_stats_test_tmp.project_name, plt.gca(), 2)

plt.tight_layout() 
plt.savefig(FIGURES_PATH + '/total_power_vs_model_size_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['macs'] = df_stats_train_tmp['macs'] / pow(10, 6)
df_stats_train_tmp['parameters'] = df_stats_train_tmp['parameters'] / pow(10, 6)
df_stats_train_tmp['trainable_params'] = df_stats_train_tmp['trainable_params'] / pow(10, 6)

sns.set(style="whitegrid")
sns.scatterplot(x="macs", y="parameters", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Multiply–Accumulate Operations as a function of the no. of parameters')
plt.xlabel("MACs (no. in millions)")
plt.ylabel("Parameters (no. in millions)")
plt.legend(loc='upper left', title='Experiment')
add_label(df_stats_train_tmp.macs, df_stats_train_tmp.parameters, df_stats_train_tmp.project_name, plt.gca(), 2) 

plt.savefig(FIGURES_PATH + '/macs_vs_no_parameters_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

sns.set(style="whitegrid")
sns.scatterplot(x="macs", y="trainable_params", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Multiply–Accumulate Operations as a function of the no. of trainable parameters')
plt.xlabel("MACs (no. in millions)")
plt.ylabel("Trainable Parameters (no. in millions)")
plt.legend(loc='upper left', title='Experiment')
add_label(df_stats_train_tmp.macs, df_stats_train_tmp.trainable_params, df_stats_train_tmp.project_name, plt.gca(), 2) 

plt.savefig(FIGURES_PATH + '/macs_vs_no_trainable_parameters_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()


In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['total'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000*EPOCHS / 3600)
df_stats_train_tmp['macs'] = df_stats_train_tmp['macs'] / pow(10, 6)
df_stats_train_tmp['parameters'] = df_stats_train_tmp['parameters'] / pow(10, 6)
df_stats_train_tmp['trainable_params'] = df_stats_train_tmp['trainable_params'] / pow(10, 6)

sns.set(style="whitegrid")
sns.scatterplot(x="total", y="macs", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Total power consumption as a function of the Multiply–Accumulate Operations (training)')
plt.xlabel("Total power consumption (Wh)")
plt.ylabel("MACs (no. in millions)")
plt.legend(loc='upper left', title='Experiment')
add_label(df_stats_train_tmp.total, df_stats_train_tmp.macs, df_stats_train_tmp.project_name, plt.gca(), 2) 

plt.savefig(FIGURES_PATH + '/total_power_vs_macs_training_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test.groupby('project_name').mean().reset_index()
df_stats_test_tmp['total'] = (df_stats_test_tmp['cpu_delta_power_w'] + (df_stats_test_tmp['gpu_power_w']/1000) + df_stats_test_tmp['ram_power_w']) * (df_stats_test_tmp['duration(s)']/1000*EPOCHS / 3600)

sns.set(style="whitegrid")
sns.scatterplot(x="total", y="macs", hue="project_name", data=df_stats_test_tmp, style='project_name', s=150)
plt.title('Total power consumption as a function of the Multiply–Accumulate Operations (testing)')
plt.xlabel("Total power consumption (Wh)")
plt.ylabel("MACs (no. in millions)")
plt.legend(loc='upper left', title='Experiment')
add_label(df_stats_test_tmp.total, df_stats_test_tmp.macs, df_stats_test_tmp.project_name, plt.gca(), 2) 

plt.savefig(FIGURES_PATH + '/total_power_vs_macs_testing_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['total'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000*EPOCHS / 3600)
df_stats_train_tmp['macs'] = df_stats_train_tmp['macs'] / pow(10, 6)
df_stats_train_tmp['parameters'] = df_stats_train_tmp['parameters'] / pow(10, 6)
df_stats_train_tmp['trainable_params'] = df_stats_train_tmp['trainable_params'] / pow(10, 6)

df_stats_train_tmp['macs_param'] = df_stats_train_tmp['macs'] / df_stats_train_tmp['trainable_params']


sns.set(style="whitegrid")
sns.scatterplot(x="total", y="macs_param", hue="project_name", data=df_stats_train_tmp, style='project_name', s=150)
plt.title('Total power consumption as a function of the MACs per parameter')
plt.xlabel("Total power consumption (Wh)")
plt.ylabel("MACs per parameter (no. in millions)")
plt.legend(loc='upper left', title='Experiment')
add_label(df_stats_train_tmp.total, df_stats_train_tmp.macs_param, df_stats_train_tmp.project_name, plt.gca(), 2) 

plt.savefig(FIGURES_PATH + '/total_power_vs_macs_per_parameter_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_train_tmp = df_stats_train
df_stats_train_tmp['total'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000 / 3600)

sns.set(style="whitegrid")
sns.lineplot(x="epoch", y="total", hue="project_name", data=df_stats_train_tmp, style='project_name')
plt.title('Average power consumption as a function of the number of epochs (training)')
plt.xlabel("Epochs")
plt.ylabel("Total power consumption (Wh)")
plt.legend(loc='lower left', title='Experiment')

plt.savefig(FIGURES_PATH + '/avg_energy_consumed_per_epoch_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_train_tmp = df_stats_train
df_stats_train_tmp['line_power'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000)
df_stats_train_tmp['total'] = df_stats_train_tmp.groupby('project_name')['line_power'].cumsum() / 3600

sns.set(style="whitegrid")
sns.lineplot(x="epoch", y="total", hue="project_name", data=df_stats_train_tmp, style='project_name')
plt.title('Average power consumption as a function of the number of epochs (training)')
plt.xlabel("Epochs")
plt.ylabel("Total power consumption (Wh)")
plt.legend(loc='upper left', title='Experiment')

plt.savefig(FIGURES_PATH + '/total_energy_consumed_per_epoch_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
df_stats_test_tmp = df_stats_test

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
line_plot = sns.lineplot(x="epoch", y="accuracy", hue="project_name", data=df_stats_test_tmp, style='project_name')
plt.title('Epochs and accuracy')
plt.xlabel("Epochs (no.)")
plt.ylabel("Accuracy (%)")
plt.legend(loc='lower left', title='Experiment')

unique_project_names = merged_df['project_name'].unique()
iterator = 0
for line in line_plot.lines:
    line_name = line.get_label()
    if "child" in line_name:
        x = line.get_xdata()
        y = line.get_ydata()

        if x.size > 0 and y.size > 0:
            t = line_plot.text(x[-1], y[-1], unique_project_names[iterator], horizontalalignment='left', size='small', color=line.get_color())
            t.set_bbox(dict(facecolor='white', edgecolor='black'))
        
        iterator += 1

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/epoch_vs_accuracy_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

df_stats_test_tmp = df_stats_test

sns.set(rc={'figure.figsize':(11.7,6.27)}, style="whitegrid")
line_plot = sns.lineplot(x="epoch", y="accuracy", hue="project_name", data=df_stats_test_tmp, style='project_name')
plt.title('Epochs and accuracy (zoomed in)')
plt.xlabel("Epochs (no.)")
plt.ylabel("Accuracy (%)")
plt.legend(loc='lower left', title='Experiment')
plt.xlim(80, None)
plt.ylim(65, None)

unique_project_names = merged_df['project_name'].unique()
iterator = 0
for line in line_plot.lines:
    line_name = line.get_label()
    if "child" in line_name:
        x = line.get_xdata()
        y = line.get_ydata()

        if x.size > 0 and y.size > 0:
            t = line_plot.text(x[-1], y[-1], unique_project_names[iterator], horizontalalignment='left', size='small', color=line.get_color())
            t.set_bbox(dict(facecolor='white', edgecolor='black'))
        
        iterator += 1

plt.tight_layout()
plt.savefig(FIGURES_PATH + '/epoch_vs_accuracy_zoom_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_df = df_stats_train
merged_df = merged_df.sort_values(['project_name', 'epoch'])
merged_df['line_power'] = (merged_df['cpu_delta_power_w'] + (merged_df['gpu_power_w']/1000) + merged_df['ram_power_w']) * (merged_df['duration(s)']/1000)
merged_df['total'] = merged_df.groupby('project_name')['line_power'].cumsum() / 3600
merged_df = merged_df[['project_name', 'epoch', 'total']]
merged_df = pd.merge(merged_df, df_stats_test, on=['project_name', 'epoch'])

sns.set(style="whitegrid")
line_plot = sns.lineplot(x="accuracy", y="total", hue="project_name", data=merged_df, style='project_name')
plt.legend(loc='upper left', title='Experiment')
plt.title('Average power consumption as a function of the accuracy')
plt.ylabel("Total power consumption (Wh)")
plt.xlabel("Accuracy (%)")
plt.xlim(70, None)

unique_project_names = merged_df['project_name'].unique()
iterator = 0
for line in line_plot.lines:
    line_name = line.get_label()
    if "child" in line_name:
        x = line.get_xdata()
        y = line.get_ydata()

        if x.size > 0 and y.size > 0:
            t = line_plot.text(x[-1], y[-1], unique_project_names[iterator], horizontalalignment='left', size='large', color=line.get_color())
            t.set_bbox(dict(facecolor='white', edgecolor='black'))
        
        iterator += 1

plt.savefig(FIGURES_PATH + '/avg_energy_consumed_vs_accuracy_per_epoch' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()


In [ ]:
merged_df = df_stats_train
# merged_df = merged_df.sort_values(['project_name', 'epoch'])
merged_df['line_power'] = (merged_df['cpu_delta_power_w'] + (merged_df['gpu_power_w']/1000) + merged_df['ram_power_w']) * (merged_df['duration(s)']/1000)
merged_df['total'] = merged_df.groupby('project_name')['line_power'].cumsum() / 3600

# Convert 'project_name' to ordinal
unique_projects = merged_df['project_name'].unique()
project_dict = {project: i for i, project in enumerate(unique_projects)}
project_ids = merged_df['project_name'].map(project_dict)

# Define grid
xi = np.linspace(project_ids.min(), project_ids.max(), 100)
yi = np.linspace(merged_df['total'].min(), merged_df['total'].max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate
zi = griddata((project_ids, merged_df['total']), merged_df['accuracy'], (xi, yi), method='linear')

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot surface
surf = ax.plot_surface(xi, yi, zi, cmap=mpl.colormaps['hot_r'])

# Labeling the projects
ax.set_xticks(range(len(unique_projects)))
ax.set_xticklabels(unique_projects, rotation=45, ha='right', fontsize=12) 

# Setting labels
ax.set_ylabel("Total power consumption (Wh)", fontsize=12)
ax.set_zlabel('Accuracy (%)', fontsize=12)

# Adding a color bar
cbar = fig.colorbar(surf)
cbar.ax.tick_params(labelsize=16)  # Set colorbar label size

plt.tight_layout()  # This will ensure that the labels don't overlap
plt.savefig(FIGURES_PATH + '/model_vs_total_power_vs_accuracy_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
merged_df = df_stats_train
merged_df = merged_df.sort_values(['project_name', 'epoch'])
merged_df['line_power'] = (merged_df['cpu_delta_power_w'] + (merged_df['gpu_power_w']/1000) + merged_df['ram_power_w']) * (merged_df['duration(s)']/1000)
merged_df['total'] = merged_df.groupby('project_name')['line_power'].cumsum() / 3600

# Convert 'project_name' to ordinal
unique_projects = merged_df['project_name'].unique()
project_dict = {project: i for i, project in enumerate(unique_projects)}
project_ids = merged_df['project_name'].map(project_dict)

# Define grid
xi = np.linspace(project_ids.min(), project_ids.max(), 100)
yi = np.linspace(merged_df['accuracy'].min(), merged_df['accuracy'].max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate
zi = griddata((project_ids, merged_df['accuracy']), merged_df['total'], (xi, yi), method='linear')

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot surface
surf = ax.plot_surface(xi, yi, zi, cmap='viridis')

# Diagonal x-ticks
ax.set_xticks(range(len(unique_projects)))
ax.set_xticklabels(unique_projects, rotation=45, ha='right')  # 45-degree angle

# Removing x-label
ax.set_xlabel('')

# Setting labels
ax.set_zlabel("Total power consumption (Wh)")
ax.set_ylabel('Accuracy (%)')

# Adding a color bar
fig.colorbar(surf)

plt.tight_layout()  # This will ensure that the labels don't overlap
plt.savefig(FIGURES_PATH + '/model_vs_accuracy_vs_total_power_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()


In [ ]:
def extract_energy_consumption_carbontracker():
    files = os.listdir(SERVER_PATH)
    data = []
    
    for file in files:
        if file.endswith('carbontracker_output.log'):
            file_path = os.path.join(SERVER_PATH, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
                for i in range(len(lines)):
                    if 'Actual consumption for' in lines[i]:
                        if i + 4 < len(lines):
                            energy_line = lines[i+2].strip()
                            energy_value = float(energy_line.split('\t')[1].split(' ')[0])
                            data.append((file.split('_')[0], energy_value))
                        break
    
    df = pd.DataFrame(data, columns=['project_name', 'CarbonTracker'])
    return df


df_stats_train_tmp = df_stats_train.groupby('project_name').mean().reset_index()
df_stats_train_tmp['FROST'] = (df_stats_train_tmp['cpu_delta_power_w'] + (df_stats_train_tmp['gpu_power_w']/1000) + df_stats_train_tmp['ram_power_w']) * (df_stats_train_tmp['duration(s)']/1000*EPOCHS / 3600)
df_stats_train_tmp = df_stats_train_tmp[[ 'project_name', 'FROST']]

df_stats_eco2ai_tmp = df_eco2ai_results_train.groupby('experiment_description').sum().reset_index()
df_stats_eco2ai_tmp['Eco2AI'] = df_stats_eco2ai_tmp['power_consumption(kWh)'] * 1000
df_stats_eco2ai_tmp = df_stats_eco2ai_tmp[['experiment_description', 'Eco2AI']]
df_stats_eco2ai_tmp.rename(columns={'experiment_description': 'project_name'}, inplace=True)

df_codecarbon_tmp = df_codecarbon_results_train.groupby('project_name')[['cpu_power', 'gpu_power', 'ram_power']].mean().reset_index()
df_codecarbon_tmp2 = df_codecarbon_results_train.groupby('project_name')['duration'].first().reset_index()
df_codecarbon_tmp = df_codecarbon_tmp.merge(df_codecarbon_tmp2, on='project_name', how='left')

df_carbontracker_tmp = extract_energy_consumption_carbontracker()
df_carbontracker_tmp['CarbonTracker'] = df_carbontracker_tmp['CarbonTracker'] * 1000

merged_duration_df = pd.DataFrame()
merged_duration_df['CodeCarbon'] = df_codecarbon_results_train.groupby('project_name')['energy_consumed'].max()*1000
merged_duration_df.reset_index(inplace=True)
merged_duration_df['CodeCarbon_correct'] = (df_codecarbon_tmp['cpu_power'] + df_codecarbon_tmp['gpu_power'] + df_codecarbon_tmp['ram_power']) * df_codecarbon_tmp['duration']*EPOCHS / 3600
merged_duration_df = pd.merge(merged_duration_df, df_carbontracker_tmp, on=['project_name'])
merged_duration_df = pd.merge(merged_duration_df, df_stats_eco2ai_tmp, on=['project_name'])
merged_duration_df = pd.merge(merged_duration_df, df_stats_train_tmp, on=['project_name'])


sns.set_theme(style="whitegrid", font_scale=2.5)

sns.catplot(x="project_name", y="value", hue="variable", kind="bar", errorbar=lambda x: (x.min(), x.max()), data=pd.melt(merged_duration_df, id_vars=['project_name']), height=16, aspect=2/1, legend_out=False)
plt.xticks(rotation=45)
plt.title('Watts per hour (Wh) from the different tools')
plt.xlabel("Models")
plt.ylabel("Watts per hour (Wh)")
plt.legend(loc='upper right', title='Experiment')

plt.savefig(FIGURES_PATH + '/watts_per_hour_all_tools_' + SERVER_NAME + '.png', bbox_inches='tight', format='png', dpi=400)
plt.show()